In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from torch.nn import functional as F
import random

model_name = 'flax-community/papuGaPT2'
device = 'cuda'
device = 'cpu'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

def log_probs_from_logits(logits, labels):
    logp = F.log_softmax(logits, dim=-1)
    logp_label = torch.gather(logp, 2, labels.unsqueeze(2)).squeeze(-1)
    return logp_label


def sentence_prob(sentence_txt):
    input_ids = tokenizer(sentence_txt, return_tensors='pt')['input_ids'].to(device)
    with torch.no_grad():
        output = model(input_ids=input_ids)
        log_probs = log_probs_from_logits(output.logits[:, :-1, :], input_ids[:, 1:])
        seq_log_probs = torch.sum(log_probs)
    return seq_log_probs.cpu().numpy()

sentences = [
  'To jest zwykłe polskie zdanie.',
  'This is a normal English sentence.',
  'iweryuiiu hrfw3eieur fr'
]

print ()
for s in sentences:
    print (s, sentence_prob(s))

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(



To jest zwykłe polskie zdanie. -26.38755
This is a normal English sentence. -37.866207
iweryuiiu hrfw3eieur fr -88.53769


In [ ]:
import itertools

def sentance_perm_prob(sentence_txt):
    words = sentence_txt.split()
    first_word = words[0]
    last_word = words[-1]
    words = words[1:-1]
    permutations = list(itertools.permutations(words))

    perm_prob_list = []

    for perm in permutations:
        perm = list(perm)
        perm.insert(0, first_word)
        perm.append(last_word)
        sentence = ' '.join(perm)
        prob = sentence_prob(sentence)
        perm_prob_list.append((sentence, prob))

    perm_prob_list.sort(key=lambda x: x[1], reverse=True)

    for sentence, prob in perm_prob_list:
        print(sentence, prob)




In [ ]:
sentance_perm_prob('Babuleńka miała dwa rogate koziołki.')

Babuleńka miała dwa rogate koziołki. -47.872154
Babuleńka rogate miała dwa koziołki. -53.9367
Babuleńka miała rogate dwa koziołki. -54.771545
Babuleńka dwa miała rogate koziołki. -59.53631
Babuleńka dwa rogate miała koziołki. -61.18611
Babuleńka rogate dwa miała koziołki. -62.98913


In [ ]:
sentance_perm_prob('Wiewiórki w parku zaczepiają przechodniów.')

Wiewiórki w parku zaczepiają przechodniów. -36.76458
Wiewiórki zaczepiają w parku przechodniów. -39.45515
Wiewiórki parku zaczepiają w przechodniów. -52.0282
Wiewiórki parku w zaczepiają przechodniów. -52.809307
Wiewiórki zaczepiają parku w przechodniów. -52.88163
Wiewiórki w zaczepiają parku przechodniów. -54.88536


In [ ]:
sentance_perm_prob('Mam do napisania sporo kodu dzisiaj.')

Mam sporo kodu do napisania dzisiaj. -32.671482
Mam do napisania sporo kodu dzisiaj. -35.88851
Mam sporo do napisania kodu dzisiaj. -36.844543
Mam sporo napisania kodu do dzisiaj. -39.314842
Mam sporo kodu napisania do dzisiaj. -40.192787
Mam kodu sporo do napisania dzisiaj. -40.204655
Mam napisania sporo kodu do dzisiaj. -41.18543
Mam kodu do napisania sporo dzisiaj. -42.243862
Mam sporo napisania do kodu dzisiaj. -42.65522
Mam sporo do kodu napisania dzisiaj. -44.292297
Mam do napisania kodu sporo dzisiaj. -45.677986
Mam napisania kodu sporo do dzisiaj. -47.079617
Mam kodu napisania sporo do dzisiaj. -47.745163
Mam napisania sporo do kodu dzisiaj. -48.003754
Mam kodu sporo napisania do dzisiaj. -48.31304
Mam do kodu napisania sporo dzisiaj. -48.407894
Mam kodu do sporo napisania dzisiaj. -48.616055
Mam napisania do kodu sporo dzisiaj. -49.37549
Mam do kodu sporo napisania dzisiaj. -49.44188
Mam napisania kodu do sporo dzisiaj. -51.31284
Mam do sporo kodu napisania dzisiaj. -51.358986

In [ ]:
def merge_triples(triples):
    chains = []

    pairs = [(i, j) for i, j, key in triples]

    while pairs:
        current_chain = list(pairs.pop(0))

        while True:
            extended = False

            for i, (a, b) in enumerate(pairs):
                if current_chain[-1] == a:
                    current_chain.append(b)
                    pairs.pop(i)
                    extended = True
                    break

                elif current_chain[0] == b:
                    current_chain.insert(0, a)
                    pairs.pop(i)
                    extended = True
                    break

            if not extended:
                break

        chains.append(current_chain)

    return chains

triples = [(1, 2, 'key1'), (2, 3, 'key2'), (3, 4, 'key3'), (5, 6, 'key4'), (6, 7, 'key5'), (7, 8, 'key6')]

chains = merge_triples(triples)

print(chains)


In [ ]:
import itertools

def sentence_perm_prob2(sentence_txt):
    words = sentence_txt.split()
    first_word = words[0]
    last_word = words[-1]

    treshold = -17
    similar_pairs = []


    for i in range(len(words)):
        max_pair = None
        for j in range(len(words)):
            if i != j and len(words[i])>1 and len(words[j])>1:
                pair_prob = sentence_prob(words[i] + ' ' + words[j])
                if pair_prob > treshold and (max_pair is None or pair_prob > max_pair[2]):
                    max_pair = (i, j, pair_prob)

        if max_pair:
            similar_pairs.append(max_pair)

    similar_pairs_sorted = sorted(similar_pairs, key=lambda x: x[2], reverse=True)

    for pair in similar_pairs_sorted:
        print(f"Para: {words[pair[0]]} {words[pair[1]]}, Prawdopodobieństwo: {pair[2]}")


    duplicates=[]
    for i in range(len(similar_pairs_sorted)):
      for j in range(i+1, len(similar_pairs_sorted)):
        if (similar_pairs_sorted[i][0]==similar_pairs_sorted[j][0] or similar_pairs_sorted[i][1]==similar_pairs_sorted[j][1]):
            duplicates.append(j)

    similar_pairs = [item for index, item in enumerate(similar_pairs_sorted) if index not in duplicates]
    similar_pairs=merge_triples(similar_pairs)
    words_chunks=[]
    all_used=[]

    for j in similar_pairs:
      chunk=[]
      for i in j:
        chunk.append(words[i])
        all_used.append(words[i])
      words_chunks.append(' '.join(chunk))

    new_bigwords=[]
    remaining_words=list(filter(lambda x: x not in all_used, words))
    if (last_word in remaining_words):
      remaining_words.remove(last_word)

    for i in words_chunks:
      if i.split()[0]==first_word:
        new_bigwords.append(i)
        words_chunks.remove(i)
        if (first_word in remaining_words):
          remaining_words.remove(first_word)
    new_bigwords+=remaining_words

    happened=0
    for i in words_chunks:
      if i.split()[-1]==last_word:
        happened=1
        words_chunks.remove(i)
        new_bigwords+=words_chunks
        new_bigwords.append(i)
        break


    if happened==0:
      new_bigwords+=words_chunks
      new_bigwords.append(last_word)



    return new_bigwords

In [ ]:
res=sentence_perm_prob2('Wczoraj wieczorem spotkałem pewną wspaniałą kobietę, która z pasją opowiadała o modelach językowych')

Para: Wczoraj wieczorem, Prawdopodobieństwo: -3.4624459743499756
Para: kobietę, która, Prawdopodobieństwo: -7.233433246612549
Para: która opowiadała, Prawdopodobieństwo: -10.009255409240723
Para: wspaniałą kobietę,, Prawdopodobieństwo: -10.627364158630371
Para: pewną pasją, Prawdopodobieństwo: -12.615326881408691
Para: pasją która, Prawdopodobieństwo: -15.735908508300781
Para: modelach językowych, Prawdopodobieństwo: -15.765219688415527
Para: wieczorem spotkałem, Prawdopodobieństwo: -16.668071746826172
Para: językowych pasją, Prawdopodobieństwo: -16.88479232788086
sim pairs sorted [(0, 1, array(-3.462446, dtype=float32)), (5, 6, array(-7.2334332, dtype=float32)), (6, 9, array(-10.009255, dtype=float32)), (4, 5, array(-10.627364, dtype=float32)), (3, 8, array(-12.615327, dtype=float32)), (8, 6, array(-15.7359085, dtype=float32)), (11, 12, array(-15.76522, dtype=float32)), (1, 2, array(-16.668072, dtype=float32)), (12, 8, array(-16.884792, dtype=float32))]
duplicates [5, 8]
sim pairs sor

In [ ]:
res

['Wczoraj wieczorem spotkałem',
 'z',
 'o',
 'wspaniałą kobietę, która opowiadała',
 'pewną pasją',
 'modelach językowych']

In [ ]:
words=res
first_word = words[0]
last_word = words[-1]
words = words[1:-1]
permutations = list(itertools.permutations(words))

perm_prob_list = []

for perm in permutations:
        perm = list(perm)
        perm.insert(0, first_word)
        perm.append(last_word)
        sentence = ' '.join(perm)
        prob = sentence_prob(sentence)
        perm_prob_list.append((sentence, prob))

perm_prob_list.sort(key=lambda x: x[1], reverse=True)

for sentence, prob in perm_prob_list:
        print(sentence, prob)

Wczoraj wieczorem spotkałem wspaniałą kobietę, która opowiadała z pewną pasją o modelach językowych -65.14698
Wczoraj wieczorem spotkałem z pewną pasją wspaniałą kobietę, która opowiadała o modelach językowych -69.48961
Wczoraj wieczorem spotkałem wspaniałą kobietę, która opowiadała o z pewną pasją modelach językowych -76.20907
Wczoraj wieczorem spotkałem o wspaniałą kobietę, która opowiadała z pewną pasją modelach językowych -84.4624
Wczoraj wieczorem spotkałem pewną pasją wspaniałą kobietę, która opowiadała o z modelach językowych -84.66511
Wczoraj wieczorem spotkałem wspaniałą kobietę, która opowiadała pewną pasją o z modelach językowych -84.90758
Wczoraj wieczorem spotkałem wspaniałą kobietę, która opowiadała o pewną pasją z modelach językowych -86.83674
Wczoraj wieczorem spotkałem wspaniałą kobietę, która opowiadała pewną pasją z o modelach językowych -87.40142
Wczoraj wieczorem spotkałem pewną pasją z wspaniałą kobietę, która opowiadała o modelach językowych -87.40903
Wczoraj wie

In [ ]:
res=sentence_perm_prob2('Bardzo dużo ludzi kupuje kamienie szlachetne, ale przecież to ogromna strata pieniędzy')

Para: ale to, Prawdopodobieństwo: -3.560964584350586
Para: Bardzo dużo, Prawdopodobieństwo: -3.982808828353882
Para: to przecież, Prawdopodobieństwo: -8.428681373596191
Para: przecież to, Prawdopodobieństwo: -9.125630378723145
Para: szlachetne, ale, Prawdopodobieństwo: -9.283679962158203
Para: strata to, Prawdopodobieństwo: -9.899646759033203
Para: ludzi to, Prawdopodobieństwo: -11.406978607177734
Para: dużo ludzi, Prawdopodobieństwo: -11.496063232421875
Para: pieniędzy to, Prawdopodobieństwo: -13.878985404968262
Para: kamienie to, Prawdopodobieństwo: -14.648480415344238
Para: kupuje to, Prawdopodobieństwo: -15.84277057647705
sim pairs sorted [(6, 8, array(-3.5609646, dtype=float32)), (0, 1, array(-3.9828088, dtype=float32)), (8, 7, array(-8.428681, dtype=float32)), (7, 8, array(-9.12563, dtype=float32)), (5, 6, array(-9.28368, dtype=float32)), (10, 8, array(-9.899647, dtype=float32)), (2, 8, array(-11.406979, dtype=float32)), (1, 2, array(-11.496063, dtype=float32)), (11, 8, array(-13

In [ ]:
res

['Bardzo dużo ludzi',
 'kupuje',
 'kamienie',
 'ogromna',
 'strata',
 'szlachetne, ale to przecież',
 'pieniędzy']

In [ ]:
words=res
first_word = words[0]
last_word = words[-1]
words = words[1:-1]
permutations = list(itertools.permutations(words))

perm_prob_list = []

for perm in permutations:
        perm = list(perm)
        perm.insert(0, first_word)
        perm.append(last_word)
        sentence = ' '.join(perm)
        prob = sentence_prob(sentence)
        perm_prob_list.append((sentence, prob))

perm_prob_list.sort(key=lambda x: x[1], reverse=True)

for sentence, prob in perm_prob_list:
        print(sentence, prob)

Bardzo dużo ludzi kupuje kamienie szlachetne, ale to przecież ogromna strata pieniędzy -50.43991
Bardzo dużo ludzi kupuje kamienie szlachetne, ale to przecież strata ogromna pieniędzy -55.16678
Bardzo dużo ludzi kupuje ogromna kamienie szlachetne, ale to przecież strata pieniędzy -61.396873
Bardzo dużo ludzi ogromna kupuje kamienie szlachetne, ale to przecież strata pieniędzy -64.52615
Bardzo dużo ludzi kamienie kupuje szlachetne, ale to przecież ogromna strata pieniędzy -69.16112
Bardzo dużo ludzi kamienie kupuje szlachetne, ale to przecież strata ogromna pieniędzy -73.14443
Bardzo dużo ludzi ogromna strata kupuje kamienie szlachetne, ale to przecież pieniędzy -75.89797
Bardzo dużo ludzi kupuje szlachetne, ale to przecież kamienie ogromna strata pieniędzy -76.08854
Bardzo dużo ludzi kupuje kamienie ogromna szlachetne, ale to przecież strata pieniędzy -76.418625
Bardzo dużo ludzi kupuje ogromna strata kamienie szlachetne, ale to przecież pieniędzy -76.98062
Bardzo dużo ludzi kamienie s

In [ ]:
res=sentence_perm_prob2('Dzień dobry moja droga przyjaciółko, chcę pożyczyć tę czerwoną sukienkę, którą ostatnio kupiłaś')
words=res
print(res)
first_word = words[0]
last_word = words[-1]
words = words[1:-1]
permutations = list(itertools.permutations(words))

perm_prob_list = []

for perm in permutations:
        perm = list(perm)
        perm.insert(0, first_word)
        perm.append(last_word)
        sentence = ' '.join(perm)
        prob = sentence_prob(sentence)
        perm_prob_list.append((sentence, prob))

perm_prob_list.sort(key=lambda x: x[1], reverse=True)

for sentence, prob in perm_prob_list:
        print(sentence, prob)

Para: Dzień dobry, Prawdopodobieństwo: -4.132632255554199
Para: moja droga, Prawdopodobieństwo: -5.2572245597839355
Para: dobry droga, Prawdopodobieństwo: -8.590896606445312
Para: tę którą, Prawdopodobieństwo: -8.9304780960083
Para: czerwoną sukienkę,, Prawdopodobieństwo: -9.017252922058105
Para: ostatnio moja, Prawdopodobieństwo: -9.221725463867188
Para: którą ostatnio, Prawdopodobieństwo: -9.934356689453125
Para: pożyczyć tę, Prawdopodobieństwo: -10.57879638671875
Para: droga którą, Prawdopodobieństwo: -14.28562068939209
Para: chcę pożyczyć, Prawdopodobieństwo: -15.522771835327148
Para: przyjaciółko, moja, Prawdopodobieństwo: -16.68511199951172
sim pairs sorted [(0, 1, array(-4.1326323, dtype=float32)), (2, 3, array(-5.2572246, dtype=float32)), (1, 3, array(-8.590897, dtype=float32)), (7, 10, array(-8.930478, dtype=float32)), (8, 9, array(-9.017253, dtype=float32)), (11, 2, array(-9.221725, dtype=float32)), (10, 11, array(-9.934357, dtype=float32)), (6, 7, array(-10.578796, dtype=flo